# Technical Factor - MASS Index (MASS)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        high - low AS DIF,
        m_ta_ema(DIF, 9) AS AHL,
        m_ta_ema(AHL, 9) AS BHL,
        AHL / BHL AS _RATIO,
        m_ta_sum(_RATIO, 25) AS MASS,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(MASS > 0 AND (MASS - m_lag(MASS, 1)) > 0, 1, 0) AS TRBY1,
        IF(MASS > 0 AND (MASS - m_lag(MASS, 1)) < 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF((MASS - m_lag(MASS, 1)) > 0 AND ((MASS - m_lag(MASS, 1)) - (m_lag(MASS, 1) - m_lag(MASS, 2))) > 0, 1, 0) AS MTBY1,
        IF((MASS - m_lag(MASS, 1)) < 0 AND ((MASS - m_lag(MASS, 1)) - (m_lag(MASS, 1) - m_lag(MASS, 2))) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(MASS > 27 AND (MASS - m_lag(MASS, 1)) < 0, 1, 0) AS RVBY1,
        IF(MASS < 25 AND (MASS - m_lag(MASS, 1)) > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(MASS, 1) <= 27 AND MASS > 27, 1, 0) AS BKBY1,
        IF(m_lag(MASS, 1) >= 25 AND MASS < 25, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        DIF,
        AHL,
        BHL,
        MASS,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,DIF,AHL,BHL,MASS,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-03-07,600519.SH,206.580292,229.884805,228.633971,25.265358,0,0,0,0,0,0,0,0
1,2025-03-10,600519.SH,168.946185,217.697081,226.446593,25.275259,1,0,0,0,0,0,0,0
2,2025-03-11,600519.SH,298.792000,233.916065,227.940487,25.389624,1,0,1,0,0,0,0,0
3,2025-03-12,600519.SH,190.044094,225.141670,227.380724,25.338817,0,1,0,1,0,0,0,0
4,2025-03-13,600519.SH,200.552318,220.223800,225.949339,25.309771,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2025-12-25,600519.SH,152.890555,112.584403,116.689068,24.075402,1,0,1,0,0,1,0,0
199,2025-12-26,600519.SH,77.205507,105.508624,114.452979,23.968832,0,1,0,1,0,0,0,0
200,2025-12-29,600519.SH,110.909005,106.588700,112.880124,23.876326,0,1,0,0,0,0,0,0
201,2025-12-30,600519.SH,134.307172,112.132394,112.730578,23.887106,1,0,1,0,0,1,0,0
